# Oracle SQL - Visualizations

In [200]:
import pandas as pd
import cx_Oracle

dsn = cx_Oracle.makedsn("srv2.lan", 1521, service_name="free")
connection = cx_Oracle.connect("hr", "oracle", dsn)

class SimpleStyler:
    def __init__(self):
        self.cells = {}
        self.rows = {}
        self.cols = {}
        self.blocks = []

        self.presets = {
            "processed-row": {'bg': '#f0f0f0', 'fg': '#000000'},
            "over":          {'bg': '#ADD8E6', 'fg': '#000000'},
            "over-matched":       {'bg': '#87CEEB', 'fg': '#000000'},
            "over-partition-col":     {'bg': '#00BFFF', 'fg': '#000000'},
            "partition":     {'bg': '#afdf81', 'fg': '#000000'},
            "subrange":      {'bg': '#dcedc8', 'fg': '#000000'},
            "result":        {'bg': '#ADFF2F', 'fg': '#000000'},
            "parameter":     {'bg': '#87CEEB', 'fg': '#000000'},       
            "keep":     {'bg': '#FFE4E1', 'fg': '#000000'},  
            "keep-matched":     {'bg': '#F08080', 'fg': '#000000'},     
        }

        self.border_styles = {}

    def resolve_colors(self, bg, fg, preset):
        if preset:
            preset_colors = self.presets.get(preset)
            if preset_colors:
                bg = preset_colors['bg']
                fg = preset_colors['fg']
        return bg or '', fg or ''

    def add_cell(self, row_idx, col_name, bg=None, fg=None, preset=None):
        bg, fg = self.resolve_colors(bg, fg, preset)
        self.cells[(row_idx, col_name)] = (bg, fg)

    def add_row(self, row_idx, bg=None, fg=None, preset=None, col_range=None):
        bg, fg = self.resolve_colors(bg, fg, preset)
        self.rows[row_idx] = (bg, fg, col_range)

    def add_column(self, col_name, bg=None, fg=None, preset=None):
        bg, fg = self.resolve_colors(bg, fg, preset)
        self.cols[col_name] = (bg, fg)

    def clear(self):
        self.cells.clear()
        self.rows.clear()
        self.cols.clear()
        self.blocks.clear()

    def style(self, df):
        styles = pd.DataFrame('', index=df.index, columns=df.columns)

        for (i, col), (bg, fg) in self.cells.items():
            if i in df.index and col in df.columns:
                css = f'background-color: {bg}; color: {fg};'
                styles.loc[i, col] += css

        for i, (bg, fg, col_range) in self.rows.items():
            if i in df.index:
                if isinstance(col_range, tuple):
                    col_range = df.columns[slice(*col_range)]
                else:
                    col_range = df.columns if col_range is None else col_range
                for col in col_range:
                    if col in df.columns:
                        css = f'background-color: {bg}; color: {fg};'
                        styles.loc[i, col] += css

        for col, (bg, fg) in self.cols.items():
            if col in df.columns:
                for i in df.index:
                    css = f'background-color: {bg}; color: {fg};'
                    styles.loc[i, col] += css

        return styles

styler = SimpleStyler()
c = styler.add_cell
r = styler.add_row
col = styler.add_column
    

## Over 
**SUM(D.salary) OVER () AS result**

In [188]:
query = """
WITH DATA (emp_id, NAME, dep_id, exp_level, salary ) AS (
  VALUES (1, 'Anna', 1, 1,1000),
         (2, 'Krystyna', 2, 2,5000 ),
         (3, 'Jan', 1, 1, 1000),
         (4, 'Zbigniew',2, 3,6000)
)
SELECT 
   D.*,
   SUM(D.salary) OVER () AS result
FROM 
DATA D
"""
df_init = pd.read_sql(query, con=connection)
df =  df_init.head()
styler.clear()

r(2, col_range=(0,5), preset="processed-row")
col("SALARY", preset="over-matched")
c(2,"RESULT", preset="result")

df.style.apply(lambda _: styler.style(df), axis=None)


C:\Users\Arek\AppData\Local\Temp\ipykernel_7996\118873147.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_init = pd.read_sql(query, con=connection)


,EMP_ID,NAME,DEP_ID,EXP_LEVEL,SALARY,RESULT
0,1,Anna,1,1,1000,13000
1,2,Krystyna,2,2,5000,13000
2,3,Jan,1,1,1000,13000
3,4,Zbigniew,2,3,6000,13000


**SUM(D.salary) OVER (partition by dep_id) AS result**

In [189]:
query = """
WITH DATA (emp_id, NAME, dep_id, exp_level, salary ) AS (
  VALUES (1, 'Anna', 1, 1,1000),
         (2, 'Krystyna', 2, 2,5000 ),
         (3, 'Jan', 1, 1, 1000),
         (4, 'Zbigniew',2, 3,6000)
)
SELECT 
   D.*,
   SUM(D.salary) OVER (partition by dep_id) AS result
FROM 
DATA D
"""
df_init = pd.read_sql(query, con=connection)
df =  df_init.head()
styler.clear()
# current row
c(2,"EMP_ID", preset="processed-row")
c(2,"NAME", preset="processed-row")
c(2,"EXP_LEVEL", preset="processed-row")
# Over
c(0,"SALARY", preset="over")
c(1,"SALARY", preset="over")
c(2,"DEP_ID", preset="over-partition-col")
c(2,"SALARY", preset="over-matched")
c(3,"SALARY", preset="over-matched")
# res
c(2,"RESULT", preset="result")

df.style.apply(lambda _: styler.style(df), axis=None)

C:\Users\Arek\AppData\Local\Temp\ipykernel_7996\3698018541.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_init = pd.read_sql(query, con=connection)  # poprawne użycie `con=`


,EMP_ID,NAME,DEP_ID,EXP_LEVEL,SALARY,RESULT
0,3,Jan,1,1,1000,2000
1,1,Anna,1,1,1000,2000
2,4,Zbigniew,2,3,6000,11000
3,2,Krystyna,2,2,5000,11000


In [205]:
query = """
WITH DATA (emp_id, NAME, dep_id, exp_level, salary ) AS (
  VALUES (1, 'Anna', 1, 1,1000),
         (2, 'Krystyna', 2, 2,5000 ),
         (3, 'Jan', 1, 1, 1000),
         (4, 'John',1, 3,7000),
          (5, 'Paul',2, 3,8000),
           (6, 'Kumar',1, 3,7000)
)
SELECT 
   D.*,
     CASE D.EXP_LEVEL WHEN 1 THEN 'Junior'
      WHEN 2 THEN 'Mid'
      WHEN 3 THEN 'Senior'
   ELSE NULL END AS explanation,
   AVG(d.salary) KEEP(DENSE_RANK LAST ORDER BY EXP_LEVEL) OVER (partition by dep_id) AS result
FROM 
DATA D
"""
df_init = pd.read_sql(query, con=connection)  # poprawne użycie `con=`
df =  df_init.head(10)
styler.clear()
# current row
c(2,"EMP_ID", preset="processed-row")
c(2,"NAME", preset="processed-row")
c(2,"EXP_LEVEL", preset="processed-row")
# Over
c(0,"SALARY", preset="over")
c(1,"SALARY", preset="over")

c(2,"DEP_ID", preset="over-partition-col")
c(2,"SALARY", preset="over-matched")
c(3,"SALARY", preset="over-matched")
#keep
c(0,"EXP_LEVEL", preset="keep")
c(1,"EXP_LEVEL", preset="keep")
c(2,"EXP_LEVEL", preset="keep-matched")
c(3,"EXP_LEVEL", preset="keep-matched")

# res
c(2,"RESULT", preset="result")

df.style.apply(lambda _: styler.style(df), axis=None)


C:\Users\Arek\AppData\Local\Temp\ipykernel_7996\3902297737.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_init = pd.read_sql(query, con=connection)  # poprawne użycie `con=`


,EMP_ID,NAME,DEP_ID,EXP_LEVEL,SALARY,EXPLANATION,RESULT
0,1,Anna,1,1,1000,Junior,7000
1,3,Jan,1,1,1000,Junior,7000
2,4,John,1,3,7000,Senior,7000
3,6,Kumar,1,3,7000,Senior,7000
4,5,Paul,2,3,8000,Senior,8000
5,2,Krystyna,2,2,5000,Mid,8000
